In [ ]:
# Client Setup
import boto3

client = boto3.client("bedrock-runtime", region_name="us-east-2")
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [11]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    # print(params)
    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [3]:
# Tool Schemas

article_details_schema = {
    "toolSpec": {
        "name": "article_details",
        "description": "This tool should be called with details about an article. It accepts information about the article's title, author, and related topics.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "The title of the article. Can be left empty.",
                    },
                    "author": {
                        "type": "string",
                        "description": "The name of the article's author. Can be left empty.",
                    },
                    "topics": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A list of topics or categories that the article covers. Can be an empty list.",
                    },
                },
            }
        },
    }
}

to_json_schema = {
    "toolSpec": {
        "name": "to_json",
        "description": "This tool processes any JSON data and can be used for generating structured content, transforming information, or creating any JSON-based output needed for your task.",
        "inputSchema": {
            "json": {"type": "object", "additionalProperties": True}
        },
    }
}

In [4]:
messages = []

add_user_message(
    messages,
    "Write a one-paragraph scholarly article about computer science. Include a title and author name",
)

result = chat(messages)

add_assistant_message(messages, result["text"])

result["text"]

'# Paradigm Shifts in Modern Computational Theory: A Brief Overview\n\nBy Dr. Eleanor J. Harrington\n\nThe evolution of computer science has been marked by significant paradigm shifts that fundamentally reshape our approach to computation and its applications across disciplines. While early computational frameworks were predominantly concerned with algorithmic efficiency measured through asymptotic analysis, contemporary research increasingly embraces multidimensional optimization that balances performance against energy consumption, ethical considerations, and human-centered design principles. This integrative approach has catalyzed breakthroughs in quantum computing, where researchers have recently demonstrated quantum advantage in specific problem domains despite ongoing challenges with qubit stability, and in artificial intelligence, where large language models exhibit emergent capabilities that transcend their explicit programming. The field now stands at a critical juncture where

In [12]:
messages = []

add_user_message(messages, 
    f"""
Analyze the article below and extract key data. Then call the article_details tool.

<article_text>
{result["text"]}
</article_text>
""",
)

# Pass the actual tool schema object, not a string
json_result = chat(messages, tools=[article_details_schema], tool_choice="article_details")
json_result

{'parts': [{'toolUse': {'toolUseId': 'tooluse_olZnRyR4QpCiuky3ZvvIGQ',
    'name': 'article_details',
    'input': {'author': 'Dr. Eleanor J. Harrington',
     'title': 'Paradigm Shifts in Modern Computational Theory: A Brief Overview',
     'topics': ['computer science',
      'computational theory',
      'quantum computing',
      'artificial intelligence',
      'multidimensional optimization',
      'interdisciplinary collaboration',
      'large language models',
      'human-centered design',
      'full-stack perspectives']}}}],
 'stop_reason': 'tool_use',
 'text': ''}